In [ ]:
import pickle

# Loading the data.

In [ ]:
features = []
targets  = []

In [ ]:
with open("data/pointing_at_camera_data", "rb") as file:
    data = pickle.load(file)

features += data[0]
targets  += [3 for i in data[1]]

## Labels: 
- 0: open hand
- 1: closed hand
- 2: swipe 
- 3: pointing at the camera

# Shuffling the data

In [ ]:
import numpy as np

In [ ]:
temp = list(zip(features, targets))
np.random.shuffle(temp)

In [ ]:
shuffled_features, shuffled_targets = zip(*temp)

# Training a simple test XGBoost

In [ ]:
from xgboost import XGBClassifier

In [ ]:
xgb_model = XGBClassifier(objective = "multi:softprob")

## Formatting the data to training

In [ ]:
features_array = np.array(shuffled_features)
features_reshaped = features_array.reshape((features_array.shape[0], features_array.shape[1]*features_array.shape[2]))

targets_array = np.array(shuffled_targets)

In [ ]:
xgb_model.fit(features_reshaped, targets_array)

# Taking a look at the shape for predicitons

In [ ]:
test_feat = features_reshaped[0]
test_target = targets_array[0]

In [ ]:
pred = xgb_model.predict_proba(test_feat.reshape(1, -1))

In [ ]:
prob = pred.max()
gestures = pred.argmax()

In [ ]:
if prob >0.9:
    print(f"Do gesture {gestures}")

# Testing live cam

In [ ]:
import cv2

from kotoamatsukami.src.services.hand_detector.service import HandDetector

In [9]:
stream = cv2.VideoCapture(0)
detector = HandDetector(max_hands = 1, detection_con = 0.8)


i = 0

while True:

    _grabbed, frame = stream.read()

    if cv2.waitKey(1) == ord("x"):
        break

    hands, image = detector.find_hands(frame)

    if len(hands) != 0:
        for hand in hands:
            feat = []

            for landmark in hand["landmarks"]:
                feat += landmark
            
            prediction = xgb_model.predict_proba([feat])
            i += 1

            if prediction.max() > 0.999:
                print(f"Catch the {prediction.argmax()} gesture - {i}")

    cv2.imshow("Video", image)

stream.release()
cv2.destroyAllWindows()

Catch the 0 gesture - 1
Catch the 0 gesture - 2
Catch the 0 gesture - 3
Catch the 0 gesture - 4
Catch the 0 gesture - 5
Catch the 0 gesture - 6
Catch the 0 gesture - 7
Catch the 0 gesture - 8
Catch the 0 gesture - 9
Catch the 0 gesture - 10
Catch the 0 gesture - 11
Catch the 0 gesture - 12
Catch the 0 gesture - 13
Catch the 0 gesture - 14
Catch the 0 gesture - 15
Catch the 0 gesture - 16
Catch the 0 gesture - 17
Catch the 0 gesture - 18
Catch the 2 gesture - 22
Catch the 2 gesture - 23
Catch the 0 gesture - 25
Catch the 0 gesture - 26
Catch the 0 gesture - 27
Catch the 0 gesture - 28
Catch the 0 gesture - 29
Catch the 0 gesture - 30
Catch the 0 gesture - 31
Catch the 0 gesture - 32
Catch the 0 gesture - 33
Catch the 0 gesture - 34
Catch the 0 gesture - 35
Catch the 0 gesture - 36
Catch the 0 gesture - 37
Catch the 0 gesture - 38
Catch the 0 gesture - 39
Catch the 0 gesture - 40
Catch the 0 gesture - 41
Catch the 0 gesture - 42
Catch the 0 gesture - 43
Catch the 0 gesture - 44
Catch the

# Saving the model

In [ ]:
with open("models/initial_4_classes_xgb_model", "wb") as file:
    pickle.dump(xgb_model, file)

# Opening the model

In [ ]:
import pickle

In [ ]:
with open("models/initial_4_classes_xgb_model", "rb") as file:
    xgb_model = pickle.load(file)